# PyTorch tutorial Neural Network

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [40]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # Convolutional Layer
        # 1 input image channel, 6 output channels, 3x3 sqr conv.
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # Linear/Dense Layer
        self.fc1 = nn.Linear(16*6*6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    @staticmethod
    def num_flat_features(x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [41]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


In [42]:
inp = torch.randn(1, 1, 32, 32)

In [43]:
out = net(inp)
print(out)

tensor([[ 0.0182, -0.0161, -0.0542, -0.1846,  0.0005,  0.0596,  0.0183,  0.0609,
         -0.0486,  0.0983]], grad_fn=<AddmmBackward>)


In [44]:
net.zero_grad()
out.backward(torch.randn(1, 10))

## Loss Function

In [45]:
output = net(inp)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.3943, grad_fn=<MseLossBackward>)


In [46]:
print(loss.grad_fn)

In [47]:
print(loss.grad_fn.next_functions[0][0])

In [48]:
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

## Backprop

In [49]:
net.zero_grad()

In [50]:
print(net.conv1.bias.grad)

tensor([0., 0., 0., 0., 0., 0.])


In [51]:
loss.backward()

In [52]:
print(net.conv1.bias.grad)

tensor([ 0.0031,  0.0122,  0.0101, -0.0159,  0.0016, -0.0041])


## Update the weights

`weight = weight - learning_rate * gradient`

In [53]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [54]:
import torch.optim as optim

In [55]:
optimizer = optim.SGD(net.parameters(), lr=0.01)

optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()